In [1]:
import glob
import os
import sys
import numpy as np
import pandas as pd
import time

import scanpy as sc
import scipy

from scipy.io import mmread
from scipy.sparse import csr_matrix
import gzip
import matplotlib.pyplot as plt
import re

import tqdm.notebook as tq

In [2]:
path = "/n/data1/hms/cellbio/sander/judy/resource_paper/"

# FrangiehIzar2021

In [3]:
index = "FrangiehIzar2021"

In [4]:
grna = pd.read_csv(index+"/"+index+"/all_sgRNA_assignments.txt")
grna.head()

,Cell,sgRNAs
0,CELL_1,HLA-B_2
1,CELL_2,"NGFR_3,SERPINF1_3"
2,CELL_3,HLA-B_2
3,CELL_4,"NMRK1_3,S100A6_3"
4,CELL_5,NaN


In [5]:
obs = pd.read_csv(index+"/"+index+"/RNA_metadata.csv", skiprows=[1])
obs.head()

,NAME,library_preparation_protocol,condition,MOI,sgRNA,UMI_count
0,CELL_1,10X 3' v3 sequencing,Control,1,HLA-B_2,10832.0
1,CELL_2,10X 3' v3 sequencing,Control,2,NaN,10731.0
2,CELL_3,10X 3' v3 sequencing,Control,1,HLA-B_2,28821.0
3,CELL_4,10X 3' v3 sequencing,Control,2,NaN,15322.0
4,CELL_5,10X 3' v3 sequencing,Control,0,NaN,10314.0


In [6]:
umap = pd.read_csv(index+"/"+index+"/RNA_UMAP_cluster.csv", skiprows=[1])

In [7]:
obs["sgRNAs"] = grna["sgRNAs"]
obs["umap_x"],obs["umap_y"] = umap["X"], umap["Y"]
obs.head()

,NAME,library_preparation_protocol,condition,MOI,sgRNA,UMI_count,sgRNAs,umap_x,umap_y
0,CELL_1,10X 3' v3 sequencing,Control,1,HLA-B_2,10832.0,HLA-B_2,-3.937523,5.441272
1,CELL_2,10X 3' v3 sequencing,Control,2,NaN,10731.0,"NGFR_3,SERPINF1_3",-5.480882,6.598254
2,CELL_3,10X 3' v3 sequencing,Control,1,HLA-B_2,28821.0,HLA-B_2,-1.044962,9.840532
3,CELL_4,10X 3' v3 sequencing,Control,2,NaN,15322.0,"NMRK1_3,S100A6_3",-2.588797,6.588357
4,CELL_5,10X 3' v3 sequencing,Control,0,NaN,10314.0,NaN,-0.795585,2.256979


In [8]:
obs.to_csv(index+"/"+index+"/obs_preprocessed.csv")

In [28]:
protein = pd.read_csv(index+"/"+index+"/raw_CITE_expression.csv.gz")

In [20]:
protein_normalized = pd.read_csv(index+"/"+index+"/Protein_expression.csv.gz")

In [38]:
cellnames = np.array(protein_normalized.columns[1:])
set(cellnames) - set(protein.columns[1:])

{'CELL_27337', 'CELL_34511', 'CELL_56031'}

In [39]:
protein['CELL_27337'] = 0
protein['CELL_34511'] = 0
protein['CELL_56031'] = 0

In [46]:
reordered = protein.iloc[:,[np.where(protein.columns == x)[0][0] for x in cellnames]]

In [47]:
X = csr_matrix(reordered.values)
obs = pd.read_csv(path+index+"/"+index+"/obs_preprocessed.csv",index_col=0)
var = protein[["Unnamed: 0"]]
adata = sc.AnnData(X.T, var=var, obs=obs)
adata.var.columns = ["Protein"]
adata.write(index+"/"+index+"/"+index+"_protein_preprocessed.h5")

Transforming to str index.
Transforming to str index.
... storing 'library_preparation_protocol' as categorical
... storing 'condition' as categorical
... storing 'sgRNA' as categorical
... storing 'sgRNAs' as categorical


In [ ]:
X = csr_matrix(protein.iloc[:,1:].values)
obs = pd.read_csv(path+index+"/"+index+"/obs_preprocessed.csv",index_col=0)
var = protein[["Protein"]]
adata = sc.AnnData(X.T, var=var, obs=obs)
adata.write(index+"/"+index+"/"+index+"_protein_preprocessed.h5")

In [4]:
adata = sc.read(index+"/"+index+"/"+index+"_RNA_preprocessed_ckpt.h5")

In [5]:
adata

AnnData object with n_obs × n_vars = 218331 × 22750 
    obs: 'NAME', 'library_preparation_protocol', 'condition', 'MOI', 'sgRNA', 'UMI_count', 'sgRNAs', 'umap_x', 'umap_y'
    var: 'gene'

In [44]:
!zcat {index+"/"+index+"/RNA_expression.csv.gz"} | wc -l

23713


In [54]:
!zcat {index+"/"+index+"/RNA_expression.csv.gz"} | tail -962 > {index+"/"+index+"/RNA_expression_tail962.csv.gz"}

In [6]:
chk=100
dtypesdict = dict(zip(range(218331),["str"]+["float"]*218331))

In [10]:
count = 0
df = pd.DataFrame()
ctime = time.time()
#for chunk in pd.read_csv(index+"/"+index+"/RNA_expression.csv.gz",chunksize=chk):
for chunk in pd.read_csv(index+"/"+index+"/RNA_expression_tail962.csv",chunksize=chk,header=None,
                         dtype = dtypesdict):
#for chunk in pd.read_csv(index+"/"+index+"/RNA_expression.csv.gz",chunksize=chk,engine='pyarrow'):
    #Xnew = csr_matrix(chunk.iloc[:,1:].values)
    #X = scipy.sparse.vstack((X,Xnew))
    df = pd.concat([df,chunk])
    count += 1
    print(time.time() - ctime)
    print(count)
    ctime = time.time()
#endtime = time.time()

206.17409133911133
1
202.77002668380737
2
202.75649619102478
3
203.12515115737915
4
204.24066472053528
5
203.87316036224365
6
203.62435960769653
7
204.81838989257812
8
206.8807282447815
9
134.4551613330841
10


In [11]:
Xfull = scipy.sparse.vstack((adata.X.T,csr_matrix(df.iloc[:,1:].values)))

In [12]:
genes = df[[0]]
genes.columns=["gene"]
var = pd.concat([adata.var,genes])
adatafull = sc.AnnData(Xfull.T,obs=adata.obs,var=var)
adatafull.write(index+"/"+index+"/"+index+"_RNA_full_preprocessed.h5")

# GasperiniShendure2019

In [ ]:
## Done!

In [3]:
index = "GasperiniShendure2019"

In [6]:
genes = pd.read_csv(index+"/lowMOI/GSE120861_pilot_lowmoi_screen.genes.txt",header=None)
genes.to_csv(index+"/lowMOI/var.csv")

In [10]:
genes = pd.read_csv(index+"/highMOI/GSE120861_pilot_highmoi_screen.genes.txt",header=None)
genes.to_csv(index+"/highMOI/var.csv")

In [11]:
genes = pd.read_csv(index+"/atscale/GSE120861_at_scale_screen.genes.txt",header=None)
genes.to_csv(index+"/atscale/var.csv")

# NormanWeissman2019

In [41]:
## Done!

In [13]:
index = "NormanWeissman2019"

In [29]:
cells = pd.read_csv(index+"/filtered/GSE133344_filtered_cell_identities.csv.gz")

In [30]:
cells

,cell_barcode,guide_identity,read_count,UMI_count,coverage,gemgroup,good_coverage,number_of_cells
0,TTGAACGAGACTCGGA-2,ARID1A_NegCtrl0__ARID1A_NegCtrl0,28684,1809,15.856274,2,True,1
1,CGTTGGGGTGTTTGTG-7,BCORL1_NegCtrl0__BCORL1_NegCtrl0,18367,896,20.498884,7,True,1
2,GAACCTAAGTGTTAGA-6,FOSB_NegCtrl0__FOSB_NegCtrl0,16296,664,24.542169,6,True,1
3,CCTTCCCTCCGTCATC-4,SET_KLF1__SET_KLF1,16262,850,19.131765,4,True,1
4,TCAATCTGTCTTTCAT-2,OSR2_NegCtrl0__OSR2_NegCtrl0,16057,1067,15.048735,2,True,2
...,...,...,...,...,...,...,...,...
111440,TTTGCGCAGTCATGCT-2,RHOXF2_NegCtrl0__RHOXF2_NegCtrl0,1,1,1.000000,2,False,0
111441,TTTGCGCCAGGACCCT-3,BCL2L11_BAK1__BCL2L11_BAK1,1,1,1.000000,3,False,0
111442,TTTGCGCGTACTTGAC-3,CNN1_NegCtrl0__CNN1_NegCtrl0,1,1,1.000000,3,False,0
111443,TTTGCGCTCTCGCATC-6,CEBPB_OSR2__CEBPB_OSR2,1,1,1.000000,6,False,0


In [19]:
genes = pd.read_csv(index+"/filtered/GSE133344_filtered_genes.tsv.gz", header=None, sep="\t")

In [24]:
genes.columns = ["ensemble_id","gene_symbol"]
genes.to_csv(index+"/filtered/var.csv")

In [37]:
cells = pd.read_csv(index+"/raw/GSE133344_raw_cell_identities.csv.gz")
cells.to_csv(index+"/raw/obs.csv")

In [38]:
genes = pd.read_csv(index+"/raw/GSE133344_raw_genes.tsv.gz", header=None, sep="\t")

In [39]:
genes.columns = ["ensemble_id","gene_symbol"]
genes.to_csv(index+"/raw/var.csv")

# TianKampmann2019

In [43]:
index = "TianKampmann2019"

In [74]:
for celltype in ["day7neuron","iPSC"]:
    for num in [1,2]:
        sgrna = pd.read_csv(glob.glob(index+"/"+celltype+"/"+celltype+str(num)+"/*sgRNA_mapping.txt.gz")[0],sep="\t")
        sgrna.to_csv(index+"/"+celltype+"/"+celltype+str(num)+"/obs.csv")
        genes = pd.read_csv(glob.glob(index+"/"+celltype+"/"+celltype+str(num)+"/*genes.tsv.gz")[0], header=None,
                   sep="\t")
        genes.columns = ["ensemble_id","gene_symbol"]
        genes.to_csv(index+"/"+celltype+"/"+celltype+str(num)+"/var.csv")
        filename = glob.glob(index+"/"+celltype+"/"+celltype+str(num)+"/*mtx.gz*")[0]
        !mv {filename} {index+"/"+celltype+"/"+celltype+str(num)+"/counts.mtx.gz"}

# TianKampmann2021

In [2]:
index = "TianKampmann2021"

In [3]:
import collections
import scipy.io as sio
import scipy.sparse as sp_sparse
import tables

In [4]:
CountMatrix = collections.namedtuple('CountMatrix', ['feature_ref', 'barcodes', 'matrix'])
 
def get_matrix_from_h5(filename):
    with tables.open_file(filename, 'r') as f:
        mat_group = f.get_node(f.root, 'matrix')
        barcodes = f.get_node(mat_group, 'barcodes').read()
        data = getattr(mat_group, 'data').read()
        indices = getattr(mat_group, 'indices').read()
        indptr = getattr(mat_group, 'indptr').read()
        shape = getattr(mat_group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)
         
        feature_ref = {}
        feature_group = f.get_node(mat_group, 'features')
        feature_ids = getattr(feature_group, 'id').read()
        feature_names = getattr(feature_group, 'name').read()
        feature_ref['id'] = feature_ids
        feature_ref['name'] = feature_names
#         tag_keys = getattr(feature_group, '_all_tag_keys').read()
#         for key in tag_keys:
#             feature_ref[key] = getattr(feature_group, key).read()
         
        return CountMatrix(feature_ref, barcodes, matrix)

In [19]:
for experiment in ["CRISPRa","CRISPRi"]:
    for idx in ["1","2","3","4"]:
        filename = glob.glob(path+index+"/"+experiment+"/"+experiment+idx+"/GSM*h5")
        if len(filename) == 1:
            filtered_feature_bc_matrix = get_matrix_from_h5(filename[0])
            var = pd.DataFrame(filtered_feature_bc_matrix.feature_ref)
            var['id'] = var['id'].str.decode("utf-8") 
            var['name'] = var['name'].str.decode("utf-8")
            #var.to_csv(index+"/"+crispr+"/"+crispr+str(num)+"/var.csv")
            obs = pd.DataFrame([x.decode("utf-8") for x in filtered_feature_bc_matrix.barcodes])
            obs.columns = ['barcode']
            #obs.to_csv(index+"/"+crispr+"/"+crispr+str(num)+"/obs.csv")
            adata = sc.AnnData(filtered_feature_bc_matrix.matrix.T, var = var, obs = obs)
            adata.write(path+index+"/"+experiment+"/"+experiment+idx+"/"+index+"_"+experiment+idx+"_preprocessed.h5")

Transforming to str index.
Transforming to str index.
... storing 'name' as categorical
Transforming to str index.
Transforming to str index.
... storing 'name' as categorical
Transforming to str index.
Transforming to str index.
... storing 'name' as categorical
Transforming to str index.
Transforming to str index.
... storing 'name' as categorical
Transforming to str index.
Transforming to str index.
... storing 'name' as categorical
Transforming to str index.
Transforming to str index.
... storing 'name' as categorical


# PapalexiSatija2021

In [3]:
index = "PapalexiSatija2021"

In [290]:
experiment = "eccite_arrayed"

In [91]:
cDNA = pd.read_csv(glob.glob(path+"/"+index+"/"+experiment+"/"+"GSM*cDNA*counts*")[0],sep="\t",index_col=0)

In [105]:
X = csr_matrix(cDNA.values)

In [100]:
genes = pd.DataFrame(cDNA.index)
genes.columns = ["gene"]
cells = pd.DataFrame(cDNA.columns)
cells.columns = ["cell_barcode"]

In [106]:
adata = sc.AnnData(X.T, var=genes, obs=cells)

Transforming to str index.
Transforming to str index.


In [108]:
adata.write(index+"/"+experiment+"/"+index+"_"+experiment+"_RNA_preprocessed.h5")

In [291]:
adt = pd.read_csv(glob.glob(path+"/"+index+"/"+experiment+"/"+"GSM*ADT_counts*")[0],sep="\t",index_col=0)

In [292]:
X = csr_matrix(adt.values)

In [293]:
proteins = pd.DataFrame(adt.index)
proteins.columns = ["protein"]
cells = pd.DataFrame(adt.columns)
cells.columns = ["cell_barcode"]

In [296]:
adata = sc.AnnData(X.T, var=proteins, obs=cells)
adata.write(index+"/"+experiment+"/"+index+"_"+experiment+"_protein_preprocessed.h5")

Transforming to str index.
Transforming to str index.


In [298]:
experiment = "eccite"

In [299]:
cDNA = pd.read_csv(glob.glob(path+"/"+index+"/"+experiment+"/"+"GSM*cDNA*counts*")[0],sep="\t",index_col=0)

In [300]:
X = csr_matrix(cDNA.values)

In [301]:
genes = pd.DataFrame(cDNA.index)
genes.columns = ["gene"]
cells = pd.DataFrame(cDNA.columns)
cells.columns = ["cell_barcode"]

In [302]:
adata = sc.AnnData(X.T, var=genes, obs=cells)
adata.write(index+"/"+experiment+"/"+index+"_"+experiment+"_RNA_preprocessed.h5")

Transforming to str index.
Transforming to str index.


In [303]:
adt = pd.read_csv(glob.glob(path+"/"+index+"/"+experiment+"/"+"GSM*ADT_counts*")[0],sep="\t",index_col=0)

In [305]:
X = csr_matrix(adt.values)

In [306]:
proteins = pd.DataFrame(adt.index)
proteins.columns = ["protein"]
cells = pd.DataFrame(adt.columns)
cells.columns = ["cell_barcode"]

In [307]:
adata = sc.AnnData(X.T, var=proteins, obs=cells)
adata.write(index+"/"+experiment+"/"+index+"_"+experiment+"_protein_preprocessed.h5")

Transforming to str index.
Transforming to str index.
